In [1]:
import torch
torch.linspace?

Docstring:
linspace(start, end, steps, *, out=None, dtype=None, layout=torch.strided, device=None, requires_grad=False) -> Tensor

Creates a one-dimensional tensor of size :attr:`steps` whose values are evenly
spaced from :attr:`start` to :attr:`end`, inclusive. That is, the value are:

.. math::
    (\text{start},
    \text{start} + \frac{\text{end} - \text{start}}{\text{steps}},
    \ldots,
    \text{start} + (\text{steps} - 1) * \frac{\text{end} - \text{start}}{\text{steps}},
    \text{end})


.. warning::
    Not providing a value for :attr:`steps` is deprecated. For backwards
    compatibility, not providing a value for :attr:`steps` will create a tensor
    with 100 elements. Note that this behavior is not reflected in the
    documented function signature and should not be relied on. In a future
    PyTorch release, failing to provide a value for :attr:`steps` will throw a
    runtime error.

Args:
    start (float): the starting value for the set of points
    end (float): the 

In [12]:
t0 = torch.linspace(0, 24, steps=25)
t0

tensor([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
        14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24.])

In [10]:
t0.dtype

torch.float32

In [13]:
t0.layout

torch.strided

In [25]:
input = t0.reshape(1, 1, 5, 5)
input

tensor([[[[ 0.,  1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.,  9.],
          [10., 11., 12., 13., 14.],
          [15., 16., 17., 18., 19.],
          [20., 21., 22., 23., 24.]]]])

In [35]:
filter = torch.ones(1, 1, 3, 3)
filter

tensor([[[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]]])

In [22]:
torch.nn.ConvTranspose2d?

Init signature:
torch.nn.ConvTranspose2d(
    in_channels: int,
    out_channels: int,
    kernel_size: Union[int, Tuple[int, int]],
    stride: Union[int, Tuple[int, int]] = 1,
    padding: Union[int, Tuple[int, int]] = 0,
    output_padding: Union[int, Tuple[int, int]] = 0,
    groups: int = 1,
    bias: bool = True,
    dilation: int = 1,
    padding_mode: str = 'zeros',
)
Docstring:     
Applies a 2D transposed convolution operator over an input image
composed of several input planes.

This module can be seen as the gradient of Conv2d with respect to its input.
It is also known as a fractionally-strided convolution or
a deconvolution (although it is not an actual deconvolution operation).

This module supports :ref:`TensorFloat32<tf32_on_ampere>`.

* :attr:`stride` controls the stride for the cross-correlation.

* :attr:`padding` controls the amount of implicit zero-paddings on both
  sides for ``dilation * (kernel_size - 1) - padding`` number of points. See note
  below for detail

In [52]:
upsample = torch.nn.ConvTranspose2d(1, 1, kernel_size=3, stride=1, bias=False)
upsample2 = torch.nn.ConvTranspose2d(1, 1, kernel_size=3, stride=2, bias=False)

In [53]:
upsample.weight.data = filter
print(upsample.weight.data)
upsample2.weight.data = filter


tensor([[[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]]])


In [54]:
output = upsample(input)
print(output)

tensor([[[[  0.,   1.,   3.,   6.,   9.,   7.,   4.],
          [  5.,  12.,  21.,  27.,  33.,  24.,  13.],
          [ 15.,  33.,  54.,  63.,  72.,  51.,  27.],
          [ 30.,  63.,  99., 108., 117.,  81.,  42.],
          [ 45.,  93., 144., 153., 162., 111.,  57.],
          [ 35.,  72., 111., 117., 123.,  84.,  43.],
          [ 20.,  41.,  63.,  66.,  69.,  47.,  24.]]]],
       grad_fn=<SlowConvTranspose2DBackward>)


In [55]:
print(output.detach().numpy())

[[[[  0.   1.   3.   6.   9.   7.   4.]
   [  5.  12.  21.  27.  33.  24.  13.]
   [ 15.  33.  54.  63.  72.  51.  27.]
   [ 30.  63.  99. 108. 117.  81.  42.]
   [ 45.  93. 144. 153. 162. 111.  57.]
   [ 35.  72. 111. 117. 123.  84.  43.]
   [ 20.  41.  63.  66.  69.  47.  24.]]]]


In [56]:
output2 = upsample2(input)
print(output2)

tensor([[[[ 0.,  0.,  1.,  1.,  3.,  2.,  5.,  3.,  7.,  4.,  4.],
          [ 0.,  0.,  1.,  1.,  3.,  2.,  5.,  3.,  7.,  4.,  4.],
          [ 5.,  5., 12.,  7., 16.,  9., 20., 11., 24., 13., 13.],
          [ 5.,  5., 11.,  6., 13.,  7., 15.,  8., 17.,  9.,  9.],
          [15., 15., 32., 17., 36., 19., 40., 21., 44., 23., 23.],
          [10., 10., 21., 11., 23., 12., 25., 13., 27., 14., 14.],
          [25., 25., 52., 27., 56., 29., 60., 31., 64., 33., 33.],
          [15., 15., 31., 16., 33., 17., 35., 18., 37., 19., 19.],
          [35., 35., 72., 37., 76., 39., 80., 41., 84., 43., 43.],
          [20., 20., 41., 21., 43., 22., 45., 23., 47., 24., 24.],
          [20., 20., 41., 21., 43., 22., 45., 23., 47., 24., 24.]]]],
       grad_fn=<SlowConvTranspose2DBackward>)


In [46]:
def trans_conv(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] + h - 1, X.shape[1] + w - 1))
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            Y[i:i + h, j:j + w] += X[i, j] * K
    return Y

In [49]:
input_2d = input.reshape(5, 5)

In [50]:
filter_2d = filter.reshape(3, 3)

In [51]:
trans_conv(input_2d, filter_2d)

tensor([[  0.,   1.,   3.,   6.,   9.,   7.,   4.],
        [  5.,  12.,  21.,  27.,  33.,  24.,  13.],
        [ 15.,  33.,  54.,  63.,  72.,  51.,  27.],
        [ 30.,  63.,  99., 108., 117.,  81.,  42.],
        [ 45.,  93., 144., 153., 162., 111.,  57.],
        [ 35.,  72., 111., 117., 123.,  84.,  43.],
        [ 20.,  41.,  63.,  66.,  69.,  47.,  24.]])